In [ ]:
# Yerbol Aussat (SID: 20698564)
# CS698, Assignment 3
# Kernel Logistic Regression
import numpy as np

In [ ]:
# importing and normalizing training data
trn_data = np.genfromtxt('train_X_dog_cat.csv', delimiter=',')
row_sums = trn_data.sum(axis=1)
trn_data = trn_data / row_sums[:, np.newaxis]

In [ ]:
# importing and normalizing training labels
trn_labels = np.genfromtxt('train_y_dog_cat.csv', delimiter=',')

In [ ]:
# importing and normalizing test data
test_data = np.genfromtxt('test_X_dog_cat.csv', delimiter=',')
row_sums = test_data.sum(axis=1)
test_data = test_data / row_sums[:, np.newaxis]

In [ ]:
# importing and normalizing test labels
test_labels = np.genfromtxt('test_y_dog_cat.csv', delimiter=',')

In [ ]:
def sigmoid(K, alpha):                                                        
    z = np.dot(K, alpha)
    return 1.0 / (1.0 + np.exp(-z)) 

In [ ]:
def gradient(K, y, alpha, reg_const):   
    # indices of positive and negative labels
    y_pos = np.where(y == 1) # locations of y = +1
    y_neg = np.where(y == -1) # locations of y = -1

    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(K, alpha)
    
    # gradient:
    g = 1.0/n_pos * np.dot(K[y_pos].T, (p[y_pos] - np.ones((n_pos, 1)) )) + 1.0/n_neg * np.dot(K[y_neg].T, (p[y_neg])) + 2.0 *reg_const * K.dot(alpha)
    return g                      

In [ ]:
def hessian(K, y, alpha, reg_const):  
    # indices of positive and negative labels
    y_pos = np.where(y == 1) # locations of y = +1
    y_neg = np.where(y == -1) # locations of y = -1
    
    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(K, alpha)
    
    # Hessian:
    H = 1.0/n_pos * K[y_pos].T.dot(np.diagflat(p[y_pos])).dot(np.diagflat(np.ones((n_pos, 1)) - p[y_pos])).dot(K[y_pos]) + 1.0/n_neg * K[y_neg].T.dot(np.diagflat(p[y_neg])).dot(np.diagflat(np.ones((n_neg, 1)) - p[y_neg])).dot(K[y_neg]) + 2.0 * reg_const * K
    return H


In [ ]:
from copy import deepcopy

def Newton(K, y_train, reg_const):
    max_pass = 50 # maximum number of iterations
    alpha = np.zeros((len(K), 1)) # initializing vector alpha
    
    iteration = 0
    while True:
        print "  iteration", iteration, ": ",
        alpha_prev = deepcopy(alpha)
        g = gradient(K, y_train, alpha, reg_const)
        H = hessian(K, y_train, alpha, reg_const)
        alpha = alpha_prev - 0.5*np.linalg.solve(H, g) # updating alpha in each iteration
        print np.linalg.norm(alpha - alpha_prev)
        iteration += 1 
        
        # Stopping condition
        if (np.linalg.norm(alpha - alpha_prev) < 10**-4) or (iteration >= max_pass):
            return alpha           
    return alpha

In [ ]:
# Define three kernels
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=5):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-np.linalg.norm(x-y)**2 / sigma)

In [ ]:
# Function calculates percent error of kernel logistic regression algorithm
def percent_error(kernel, trn_data, trn_labels, test_data, test_labels, lambdas, sigma=5):
    if kernel == linear_kernel:
        kernel_name = "Linear Kernel"
    elif kernel == polynomial_kernel:
        kernel_name = "Polynomial Kernel"    
    elif kernel == gaussian_kernel:
        kernel_name = "Gaussian Kernel"    
    
    # Calculate kernel matrix
    print "Calculating", kernel_name, "matrix"
    n_samples = len(trn_labels)
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            if kernel == gaussian_kernel:
                K[i,j] = kernel(trn_data[i], trn_data[j], sigma)
            else:
                K[i,j] = kernel(trn_data[i], trn_data[j])
    
    for i in range(len(lambdas)):   
        reg_const = lambdas[i]
        
        # Training kernel logistic regression algorithm for the value of lambda
        print "Training kernel logistic regression algorithm for lambda =", reg_const
        alpha_vector = Newton(K, trn_labels, reg_const)
    
        # Testing kernel logistic regression algorithm for lambda = 1
        print "Testing kernel logistic regression algorithm for lambda =", reg_const
        incorrect = 0
        for sample_i in range(len(test_labels)):
            x = test_data[sample_i]
            K0 = np.zeros(n_samples)
            for i in range(n_samples):  
                if kernel == gaussian_kernel:
                    K0[i] = kernel(trn_data[i], x, sigma)
                else:
                    K0[i] = kernel(trn_data[i], x)                
            p0 = sigmoid(K0, alpha_vector) 
            if (p0 >= 0.5):
                prediction = 1
            else:
                prediction = -1
            if prediction != test_labels[sample_i]:
                incorrect+=1
        if kernel == gaussian_kernel:
            print "\n", kernel_name, ", lambda =", reg_const, ", sigma =", sigma
        else:
            print "\n", kernel_name, ": lambda =", reg_const, ":"
        print "  Num Incorrectly Classified:", incorrect
        print "  Percent Error:", 1.0*incorrect / len(test_labels)
        print "\n", '*'*40, '\n'


In [ ]:
# LINEAR KERNEL
percent_error(linear_kernel, trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10])

In [ ]:
# POLYNOMIAL KERNEL
percent_error(polynomial_kernel, trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10])

In [ ]:
# GAUSSIAN KERNEL (sigma = 5)
percent_error(gaussian_kernel, trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10], 5)

In [ ]:
# GAUSSIAN KERNEL (sigma = 10)
percent_error(gaussian_kernel, trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10], sigma=10)

In [ ]:
# GAUSSIAN KERNEL (sigma = 1)
percent_error(gaussian_kernel, trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10], sigma=1)